<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [30]:
import pandas as pd
import csv as csv

In [31]:
# Load training data
train_df = pd.read_csv("data/train.csv", header=0)

In [32]:
train_df.head(3)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
train_label = train_df.values[:,0]
train_label

array([1, 0, 1, ..., 7, 6, 9])

In [34]:
from tensorflow.python.keras.utils import to_categorical
train_label = to_categorical(train_label)
train_label

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [35]:
train_data = train_df.values[:,1:]
train_data.reshape(42000,28,28)
train_data = train_data/255.

In [36]:
train_data = train_data.reshape(-1,28,28,1)
train_data.shape

(42000, 28, 28, 1)

In [37]:
# Load testning data
test_df = pd.read_csv("data/test.csv", header=0)
test_df.head(3)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
test_data = test_df.values
test_data.reshape(28000,28,28)
test_data = test_data/255.

In [39]:
test_data = test_data.reshape(-1,28,28,1)
test_data.shape

(28000, 28, 28, 1)

In [11]:
# とりあえず先頭の20000のみを使用
#train_data = train_data[:20000]
#train_label = train_label[:20000]

In [44]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense
model = Sequential()

In [45]:
def latest_model():
    model.add(
        Conv2D(
            filters=32,
            input_shape=(28, 28, 1),
            kernel_size=(5, 5),
            strides=(1, 1),
            padding='same',
            activation='relu'
        )
    )

    model.add(
        Conv2D(
            filters=32,
            kernel_size=(5, 5),
            strides=(1, 1),
            padding='same',
            activation='relu'
        )
    )
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Dropout(0.25))
    
    model.add(
        Conv2D(
            filters=64,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding='same',
            activation='relu'
        )
    )
    
    model.add(
        Conv2D(
            filters=64,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding='same',
            activation='relu'
        )
    )
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    
    model.add(Dense(units=256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=10, activation='softmax'))

In [46]:
latest_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 7, 7, 64)          0         
__________

In [47]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_model1 = model.fit(
    train_data,
    train_label,
    batch_size=32,
    epochs=30,
    validation_split=0.2
)

Train on 33600 samples, validate on 8400 samples
Epoch 1/30
33600/33600 [==============================] - 9s - loss: 0.2252 - acc: 0.9303 - val_loss: 0.0703 - val_acc: 0.9789
Epoch 2/30
33600/33600 [==============================] - 9s - loss: 0.0804 - acc: 0.9759 - val_loss: 0.0535 - val_acc: 0.9824
Epoch 3/30
33600/33600 [==============================] - 9s - loss: 0.0601 - acc: 0.9823 - val_loss: 0.0408 - val_acc: 0.9886
Epoch 4/30
33600/33600 [==============================] - 8s - loss: 0.0522 - acc: 0.9845 - val_loss: 0.0372 - val_acc: 0.9889
Epoch 5/30
33600/33600 [==============================] - 9s - loss: 0.0453 - acc: 0.9862 - val_loss: 0.0315 - val_acc: 0.9915
Epoch 6/30
33600/33600 [==============================] - 8s - loss: 0.0377 - acc: 0.9882 - val_loss: 0.0295 - val_acc: 0.9920
Epoch 7/30
33600/33600 [==============================] - 9s - loss: 0.0335 - acc: 0.9897 - val_loss: 0.0360 - val_acc: 0.9911
Epoch 8/30
33600/33600 [==============================] - 8s -

In [48]:
res = model.predict(test_data)

In [49]:
res[0]

array([8.3367072e-19, 3.3736287e-21, 1.0000000e+00, 2.7376869e-17,
       6.3125852e-21, 2.7052914e-28, 2.1932532e-21, 1.7542104e-16,
       1.4554159e-19, 1.8340622e-22], dtype=float32)

In [50]:
output = [0 for i in range(len(res))]
ids = [i for i in range(1,len(res)+1)]
for i in range(len(res)):
    output[i] = res[i].argmax()

In [51]:
submit_file = open("submit.csv", "w")
file_object = csv.writer(submit_file)
file_object.writerow(["ImageId", "Label"])
file_object.writerows(zip(ids, output))
submit_file.close()